In [5]:
import tensorflow as tf
print(tf.__version__)

2.0.0-rc1


# 自动微分

In [12]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x
dy_dx = g.gradient(y, x)  # y’ = 2*x = 2*3 = 6


**GradientTape**是eager模式下计算梯度用的

**watch(tensor)**

作用：确保某个tensor被tape追踪 

参数:tensor: 一个Tensor或者一个Tensor列表

**gradient(target, sources)**

作用：根据tape上面的上下文来计算某个或者某些tensor的梯度参数

target: 被微分的Tensor或者Tensor列表，你可以理解为经过某个函数之后的值

sources: Tensors 或者Variables列表（当然可以只有一个值）. 你可以理解为函数的某个变量

返回:
一个列表表示各个变量的梯度值，和source中的变量列表一一对应，表明这个变量的梯度。
上面的例子中的梯度计算部分可以更直观的理解这个函数的用法。


In [94]:
x=tf.constant(3.0)
with tf.GradientTape(persistent=True) as g:
    g.watch(x)   
    y = x * x
    z = y * y
dz_dx=g.gradient(z,x)
# z=y^2=x^4,
# z’=4*x^3=4*3^3
dy_dx=g.gradient(y,x)
# y’=2*x=2*3=6


<tf.Tensor: id=31893, shape=(), dtype=float32, numpy=6.0>

In [17]:
class MyModel(tf.keras.Model):

    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        # 定义自己需要的层
        self.dense_1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense_2 = tf.keras.layers.Dense(num_classes)

    def call(self, inputs):
        #定义前向传播
        # 使用在 (in `__init__`)定义的层
        x = self.dense_1(inputs)
        return self.dense_2(x)


In [18]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))


In [20]:
model = MyModel(num_classes=10)

loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()


with tf.GradientTape() as tape:
    predictions = model(data)
    loss = loss_object(labels, predictions)
gradients = tape.gradient(loss, model.trainable_variables)
optimizer.apply_gradients(zip(gradients, model.trainable_variables))





If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

In [ ]:
model.trainable_variables

## keras 版本模型训练

当使用内置的培训和验证API（如model.fit()，model.evaluate()，model.predict()）。在“使用内置的训练和评估循环”部分中对此进行了介绍。

### 第一部分：使用内置的训练和评估循环
让我们考虑以下模型（在这里，我们使用函数式API进行内置，但是它也可以是顺序模型或子类模型）

In [104]:
inputs = tf.keras.Input(shape=(32,))  
x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.Dense(64, activation='relu')(x)
predictions = layers.Dense(10)(x)


```python
model = tf.keras.Model(inputs=inputs, outputs=predictions)
```
- inputs(模型输入)
- output(模型输出)


```python
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
```

- 指定损失函数 (loss) tf.keras.optimizers.RMSprop
- 优化器 (optimizer) tf.keras.losses.SparseCategoricalCrossentropy
- 指标 (metrics) ['accuracy'] 

In [ ]:
model = tf.keras.Model(inputs=inputs, outputs=predictions)


model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


#### 模型训练

In [ ]:

import numpy as np
x_train = np.random.random((1000, 32))
y_train = np.random.randint(10, size=(1000, ))

x_val = np.random.random((200, 32))
y_val = np.random.randint(10, size=(200, ))
x_test = np.random.random((200, 32))
y_test = np.random.randint(10, size=(200, ))
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val))

#### model.evaluate

返回 test loss 和metrics

In [ ]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(x_test, y_test, batch_size=128)
print('test loss, test acc:', results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions for 3 samples')
predictions = model.predict(x_test[:3])
print('predictions shape:', predictions.shape)

如果模型有多个输出，则可以为每个输出指定不同的损失和指标，并且可以调制每个输出对模型总损失的贡献。


另外，如果想用上述的默认设置，那么在很多情况下，可以通过字符串标识符指定优化器，损失和指标：

In [33]:
model.compile(optimizer='rmsprop',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

为了以后的重用，让我们在函数中放入模型定义和编译步骤。我们后面会多次调用它

In [36]:
def get_uncompiled_model():
    inputs = tf.keras.Input(shape=(32,), name='digits')
    x = tf.keras.layers.Dense(64, activation='relu', name='dense_1')(inputs)
    x = tf.keras.layers.Dense(64, activation='relu', name='dense_2')(x)
    outputs = tf.keras.layers.Dense(10, name='predictions')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model


def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['sparse_categorical_accuracy'])
    return model


提供[许多内置的优化器，损失和指标](https://www.tensorflow.org/guide/keras/train_and_evaluate#many_built-in_optimizers_losses_and_metrics_are_available)
通常，您不必从头开始创建自己的损失，指标或优化程序，因为所需的可能已经是Keras API的一部分：

优化器：
- SGD() （有或没有动量）
- RMSprop()
- Adam()

损失：
- MeanSquaredError()
- KLDivergence()
- CosineSimilarity()

指标：
- AUC()
- Precision()
- Recall()


海关损失

使用Keras提供两种方式来提供自定义损失。第一个示例创建一个接受输入y_true和的函数y_pred。以下示例显示了损失函数，该函数计算实际数据和预测之间的平均绝对误差：

In [ ]:
def basic_loss_function(y_true, y_pred):
    return tf.math.reduce_mean(tf.abs(y_true - y_pred))

model.compile(optimizer=keras.optimizers.Adam(),
              loss=basic_loss_function)

model.fit(x_train, y_train, batch_size=64, epochs=3)

如果您需要一个损失函数，该函数需要一个y_true和旁边的参数y_pred，则可以对该tf.keras.losses.Loss类进行子类化并实现以下两种方法：

- `__init__(self)` -接受在损失函数调用期间传递的参数
- call(self, y_true, y_pred)—使用目标（y_true）和模型预测（y_pred）计算模型的损失
- 在计算损失时`__init__()`可以使用传入的参数call()。

以下示例显示了如何实现WeightedCrossEntropy计算BinaryCrossEntropy损失的损失函数，其中某个类或整个函数的损失可以通过标量进行修改。

In [45]:
class WeightedBinaryCrossEntropy(keras.losses.Loss):
    """
    Args:
      pos_weight: Scalar to affect the positive labels of the loss function.
      weight: Scalar to affect the entirety of the loss function.
      from_logits: Whether to compute loss from logits or the probability.
      reduction: Type of tf.keras.losses.Reduction to apply to loss.
      name: Name of the loss function.
    """
    def __init__(self, pos_weight, weight, from_logits=False,
                 reduction=keras.losses.Reduction.AUTO,
                 name='weighted_binary_crossentropy'):
        super().__init__(reduction=reduction, name=name)
        self.pos_weight = pos_weight
        self.weight = weight
        self.from_logits = from_logits

    def call(self, y_true, y_pred):
        ce = tf.losses.binary_crossentropy(
            y_true, y_pred, from_logits=self.from_logits)[:,None]
        ce = self.weight * (ce*(1-y_true) + self.pos_weight*ce*(y_true))
        return ce

这是一个二进制损失，但是数据集有10个类别，因此应应用该损失，就像模型为每个类别进行独立的二进制预测一样。为此，首先从类索引创建单热点向量

In [46]:
one_hot_y_train = tf.one_hot(y_train.astype(np.int32), depth=10)

In [47]:
one_hot_y_train

<tf.Tensor: id=10747, shape=(1000, 10), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

现在，使用那些热点，以及自定义损失来训练模型：

In [48]:
model = get_uncompiled_model()

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=WeightedBinaryCrossEntropy(
        pos_weight=0.5, weight = 2, from_logits=True)
)

model.fit(x_train, one_hot_y_train, batch_size=64, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 724us/sample - loss: 0.9656
Epoch 2/5
1000/1000 [==============================] - 0s 75us/sample - loss: 0.6662
Epoch 3/5
1000/1000 [==============================] - 0s 75us/sample - loss: 0.6349
Epoch 4/5
1000/1000 [==============================] - 0s 80us/sample - loss: 0.6250
Epoch 5/5
1000/1000 [==============================] - 0s 83us/sample - loss: 0.6226


自定义指标
如果您需要的指标不是API的一部分，则可以通过将该Metric类子类化来轻松创建自定义指标。您将需要实现4种方法：

`__init__(self)`，您将在其中为指标创建状态变量。
update_state(self, y_true, y_pred, sample_weight=None)，它使用目标y_true和模型预测y_pred来更新状态变量。
result(self)，它使用状态变量来计算最终结果。
reset_states(self)，它会重新初始化指标的状态。
状态更新和结果计算保持分开（分别在update_state()和中result()），因为在某些情况下，结果计算可能会非常昂贵，并且只能定期执行。

这是一个简单的示例，显示了如何实施CategoricalTruePositives度量标准，该度量标准统计了正确归类为给定类的样本数量：

In [50]:
class CategoricalTruePositives(keras.metrics.Metric):

    def __init__(self, name='categorical_true_positives', **kwargs):
        super(CategoricalTruePositives, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.reshape(tf.argmax(y_pred, axis=1), shape=(-1, 1))
        values = tf.cast(y_true, 'int32') == tf.cast(y_pred, 'int32')
        values = tf.cast(values, 'float32')
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, 'float32')
            values = tf.multiply(values, sample_weight)
        self.true_positives.assign_add(tf.reduce_sum(values))

    def result(self):
        return self.true_positives

    def reset_states(self):
        # The state of the metric will be reset at the start of each epoch.
        self.true_positives.assign(0.)


In [51]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[CategoricalTruePositives()])
model.fit(x_train, y_train,
          batch_size=64,
          epochs=3)

Train on 1000 samples
Epoch 1/3
1000/1000 [==============================] - 1s 688us/sample - loss: 2.3114 - categorical_true_positives: 95.0000
Epoch 2/3
1000/1000 [==============================] - 0s 73us/sample - loss: 2.2931 - categorical_true_positives: 114.0000
Epoch 3/3
1000/1000 [==============================] - 0s 75us/sample - loss: 2.2906 - categorical_true_positives: 117.0000


处理不符合标准签名的损失和指标
绝大多数损失和指标可以通过y_true和计算y_pred，其中y_pred是模型的输出。但不是所有人。例如，正则化损失可能仅需要激活层（在这种情况下没有目标），并且此激活可能不是模型输出。

在这种情况下，您可以self.add_loss(loss_value)从call自定义图层的方法内部进行调用。这是一个添加活动正则化的简单示例（请注意，活动正则化内置在所有Keras层中-该层仅是为了提供一个具体示例）：

In [55]:
class ActivityRegularizationLayer(layers.Layer):

    def call(self, inputs):
        self.add_loss(tf.reduce_sum(inputs) * 0.1)
        return inputs  # Pass-through layer.


inputs = keras.Input(shape=(32,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)

# Insert activity regularization as a layer
x = ActivityRegularizationLayer()(x)

x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True))

# The displayed loss will be much higher than before
# due to the regularization component.
model.fit(x_train, y_train,
          batch_size=64,
          epochs=1)


Train on 1000 samples
1000/1000 [==============================] - 1s 652us/sample - loss: 41.9112


您可以对记录指标值执行相同的操作：

In [58]:
class MetricLoggingLayer(layers.Layer):

    def call(self, inputs):
        # The `aggregation` argument defines
        # how to aggregate the per-batch values
        # over each epoch:
        # in this case we simply average them.
        self.add_metric(keras.backend.std(inputs),
                        name='std_of_activation',
                        aggregation='mean')
        return inputs  # Pass-through layer.


inputs = keras.Input(shape=(32,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)

# Insert std logging as a layer.
x = MetricLoggingLayer()(x)

x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True))
model.fit(x_train, y_train,
          batch_size=64,
          epochs=1)


Train on 1000 samples
1000/1000 [==============================] - 1s 697us/sample - loss: 2.3214 - std_of_activation: 0.2838


在Functional API中，您还可以调用model.add_loss(loss_tensor)或model.add_metric(metric_tensor, name, aggregation)。

这是一个简单的示例：

In [60]:
inputs = keras.Input(shape=(32,), name='digits')
x1 = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x2 = layers.Dense(64, activation='relu', name='dense_2')(x1)
outputs = layers.Dense(10, name='predictions')(x2)
model = keras.Model(inputs=inputs, outputs=outputs)

model.add_loss(tf.reduce_sum(x1) * 0.1)

model.add_metric(keras.backend.std(x1),
                 name='std_of_activation',
                 aggregation='mean')

model.compile(optimizer=keras.optimizers.RMSprop(1e-3),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True))
model.fit(x_train, y_train,
          batch_size=64,
          epochs=1)


Train on 1000 samples
1000/1000 [==============================] - 1s 910us/sample - loss: 37.5822 - std_of_activation: 0.1789


自动区分验证保留集
在您看到的第一个端到端示例中，我们使用validation_data参数将Numpy数组的元组传递(x_val, y_val)给模型，以在每个时期结束时评估验证损失和验证指标。

这是另一个选择：参数validation_split允许您自动保留部分训练数据以供验证。参数值代表要保留用于验证的数据的一部分，因此应将其设置为大于0且小于1的数字。例如，validation_split=0.2表示“使用20％的数据进行验证”，而validation_split=0.6表示“使用60％的数据用于验证”。

验证的计算方法是在进行任何改组之前，对fit调用接收到的数组进行最后x％的采样。

您只能validation_split在使用Numpy数据进行训练时使用。

In [62]:
model = get_compiled_model()
model.fit(x_train, y_train, batch_size=64, validation_split=0.2, epochs=1, steps_per_epoch=1)

Train on 800 samples, validate on 200 samples
 64/800 [=>............................] - ETA: 8s - loss: 2.3066 - sparse_categorical_accuracy: 0.0469 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 0.0000e+00

## 使用样本加权和类别加权

除了输入数据和目标数据外，还可以在使用时将样本权重或类权重传递给模型fit：

从Numpy数据进行训练时：通过sample_weight和class_weight参数。
从数据集训练时：通过使数据集返回一个元组(input_batch, target_batch, sample_weight_batch)。
“样本权重”数组是一个数字数组，用于指定批次中每个样本在计算总损失时应具有的权重。它通常用于不平衡的分类问题中（这种想法是为很少见的班级赋予更多的权重）。当所使用的权重为1和0时，该数组可用作损失函数的掩码（完全丢弃某些样本对总损失的贡献）。

“类别权重”字典是同一概念的一个更具体的实例：它将类别索引映射到应该用于属于该类别的样本的样本权重。例如，如果在数据中类“ 0”的表示量比类“ 1”的表示量少两倍，则可以使用class_weight={0: 1., 1: 0.5}。

这是一个Numpy示例，其中我们使用类权重或样本权重来更加重视5类（在MNIST数据集中的数字“ 5”）的正确分类。

In [63]:
import numpy as np

class_weight = {0: 1., 1: 1., 2: 1., 3: 1., 4: 1.,
                # Set weight "2" for class "5",
                # making this class 2x more important
                5: 2.,
                6: 1., 7: 1., 8: 1., 9: 1.}
print('Fit with class weight')
model.fit(x_train, y_train,
          class_weight=class_weight,
          batch_size=64,
          epochs=4)

Fit with class weight
Train on 1000 samples
Epoch 1/4
1000/1000 [==============================] - 1s 570us/sample - loss: 2.5203 - sparse_categorical_accuracy: 0.1000
Epoch 2/4
1000/1000 [==============================] - 0s 90us/sample - loss: 2.4901 - sparse_categorical_accuracy: 0.0910
Epoch 3/4
1000/1000 [==============================] - 0s 104us/sample - loss: 2.4807 - sparse_categorical_accuracy: 0.0910
Epoch 4/4
1000/1000 [==============================] - 0s 88us/sample - loss: 2.4714 - sparse_categorical_accuracy: 0.0960


In [65]:
# Here's the same example using `sample_weight` instead:
sample_weight = np.ones(shape=(len(y_train),))
sample_weight[y_train == 5] = 2.
print('\nFit with sample weight')

model = get_compiled_model()
model.fit(x_train, y_train,
          sample_weight=sample_weight,
          batch_size=64,
          epochs=4)


Fit with sample weight
Train on 1000 samples
Epoch 1/4
1000/1000 [==============================] - 1s 846us/sample - loss: 2.5227 - sparse_categorical_accuracy: 0.1100
Epoch 2/4
1000/1000 [==============================] - 0s 83us/sample - loss: 2.4891 - sparse_categorical_accuracy: 0.1090
Epoch 3/4
1000/1000 [==============================] - 0s 84us/sample - loss: 2.4758 - sparse_categorical_accuracy: 0.0970
Epoch 4/4
1000/1000 [==============================] - 0s 86us/sample - loss: 2.4670 - sparse_categorical_accuracy: 0.0960


### 将数据传递到多输入，多输出模型
在前面的示例中，我们正在考虑一个具有单个输入（shape的张量(764,)）和单个输出（shape的预测张量(10,)）的模型。但是具有多个输入或输出的模型呢？

考虑以下模型，该模型具有形状的图像输入(32, 32, 3)（即(height, width, channels)）和形状的时间序列输入(None, 10)（即(timesteps, features)）。我们的模型将具有根据这些输入的组合计算出的两个输出：“得分”（形状(1,)）和五类（形状(5,)）的概率分布。

In [67]:
from tensorflow import keras
from tensorflow.keras import layers

image_input = keras.Input(shape=(32, 32, 3), name='img_input')
timeseries_input = keras.Input(shape=(None, 10), name='ts_input')

x1 = layers.Conv2D(3, 3)(image_input)
x1 = layers.GlobalMaxPooling2D()(x1)

x2 = layers.Conv1D(3, 3)(timeseries_input)
x2 = layers.GlobalMaxPooling1D()(x2)

x = layers.concatenate([x1, x2])

score_output = layers.Dense(1, name='score_output')(x)
class_output = layers.Dense(5, name='class_output')(x)

model = keras.Model(inputs=[image_input, timeseries_input],
                    outputs=[score_output, class_output])

让我们绘制这个模型，以便您可以清楚地看到我们在这里做什么（请注意，图中显示的形状是批处理形状，而不是按样本的形状）。

In [70]:
keras.utils.plot_model(model, 'multi_input_and_output_model.png', show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


在编译时，通过将损失函数作为列表传递，我们可以为不同的输出指定不同的损失：

In [ ]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=[keras.losses.MeanSquaredError(),
          keras.losses.CategoricalCrossentropy(from_logits=True)])

如果我们仅将单个损失函数传递给模型，则将相同的损失函数应用于每个输出，这在此处不合适。

同样对于指标：

In [71]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=[keras.losses.MeanSquaredError(),
          keras.losses.CategoricalCrossentropy(from_logits=True)],
    metrics=[[keras.metrics.MeanAbsolutePercentageError(),
              keras.metrics.MeanAbsoluteError()],
             [keras.metrics.CategoricalAccuracy()]])

由于我们为输出层命名，因此我们还可以通过dict指定每个输出的损失和指标：

In [72]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={'score_output': keras.losses.MeanSquaredError(),
          'class_output': keras.losses.CategoricalCrossentropy(from_logits=True)},
    metrics={'score_output': [keras.metrics.MeanAbsolutePercentageError(),
                              keras.metrics.MeanAbsoluteError()],
             'class_output': [keras.metrics.CategoricalAccuracy()]})

如果您有两个以上的输出，我们建议使用显式名称和字典。

可以使用以下参数对不同的特定于输出的损失赋予不同的权重（例如，在我们的示例中，我们可能希望通过将类损失的重要性赋予2倍来对“得分”损失给予特权）loss_weights：

In [73]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={'score_output': keras.losses.MeanSquaredError(),
          'class_output': keras.losses.CategoricalCrossentropy(from_logits=True)},
    metrics={'score_output': [keras.metrics.MeanAbsolutePercentageError(),
                              keras.metrics.MeanAbsoluteError()],
             'class_output': [keras.metrics.CategoricalAccuracy()]},
    loss_weights={'score_output': 2., 'class_output': 1.})

您还可以选择不为某些输出计算损失，如果这些输出仅用于预测而不是训练：

In [74]:
# List loss version
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=[None, keras.losses.CategoricalCrossentropy(from_logits=True)])

# Or dict loss version
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={'class_output':keras.losses.CategoricalCrossentropy(from_logits=True)})

In [75]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=[keras.losses.MeanSquaredError(),
          keras.losses.CategoricalCrossentropy(from_logits=True)])

# Generate dummy Numpy data
img_data = np.random.random_sample(size=(100, 32, 32, 3))
ts_data = np.random.random_sample(size=(100, 20, 10))
score_targets = np.random.random_sample(size=(100, 1))
class_targets = np.random.random_sample(size=(100, 5))

# Fit on lists
model.fit([img_data, ts_data], [score_targets, class_targets],
          batch_size=32,
          epochs=3)

# Alternatively, fit on dicts
model.fit({'img_input': img_data, 'ts_input': ts_data},
          {'score_output': score_targets, 'class_output': class_targets},
          batch_size=32,
          epochs=3)

Train on 100 samples
Epoch 1/3
100/100 [==============================] - 3s 27ms/sample - loss: 6.6108 - score_output_loss: 1.2894 - class_output_loss: 5.3378
Epoch 2/3
100/100 [==============================] - 0s 469us/sample - loss: 5.7249 - score_output_loss: 0.6774 - class_output_loss: 4.7727
Epoch 3/3
100/100 [==============================] - 0s 389us/sample - loss: 5.3735 - score_output_loss: 0.4115 - class_output_loss: 4.8524
Train on 100 samples
Epoch 1/3
100/100 [==============================] - 0s 549us/sample - loss: 5.1592 - score_output_loss: 0.2520 - class_output_loss: 4.8538
Epoch 2/3
100/100 [==============================] - 0s 399us/sample - loss: 5.0296 - score_output_loss: 0.1793 - class_output_loss: 4.9180
Epoch 3/3
100/100 [==============================] - 0s 379us/sample - loss: 4.9313 - score_output_loss: 0.1469 - class_output_loss: 4.8615


### 使用回调
Keras中的回调是在训练期间（在某个时期开始时，在批处理结束时，在某个时期结束时等）在不同时间点调用的对象，这些对象可用于实现以下行为：

在训练过程中的不同时间点进行验证（除了内置的按时间段验证）
定期或在超过特定精度阈值时对模型进行检查
当训练似乎停滞不前时，更改模型的学习率
当训练似乎停滞不前时，对顶层进行微调
在培训结束或超出特定性能阈值时发送电子邮件或即时消息通知
等等。
回调可以作为列表传递给您的呼叫fit：

In [76]:
model = get_compiled_model()

callbacks = [
    keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor='val_loss',
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-2,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=2,
        verbose=1)
]
model.fit(x_train, y_train,
          epochs=20,
          batch_size=64,
          callbacks=callbacks,
          validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/20
800/800 [==============================] - 1s 1ms/sample - loss: 2.3268 - sparse_categorical_accuracy: 0.1088 - val_loss: 2.3227 - val_sparse_categorical_accuracy: 0.1000
Epoch 2/20
800/800 [==============================] - 0s 118us/sample - loss: 2.3030 - sparse_categorical_accuracy: 0.1150 - val_loss: 2.3320 - val_sparse_categorical_accuracy: 0.0750
Epoch 3/20
800/800 [==============================] - 0s 113us/sample - loss: 2.2909 - sparse_categorical_accuracy: 0.1325 - val_loss: 2.3271 - val_sparse_categorical_accuracy: 0.1000
Epoch 00003: early stopping


### 许多内置的回调可用
- ModelCheckpoint：定期保存模型。
- EarlyStopping：当培训不再改善验证指标时，停止培训。
- TensorBoard：定期编写可在TensorBoard中可视化的模型日志（更多详细信息，请参见“可视化”部分）。
- CSVLogger：将损失和指标数据流式传输到CSV文件。
等等

## 编写自己的回调
您可以通过扩展基类keras.callbacks.Callback来创建自定义回调。回调可以通过class属性访问其关联的模型self.model。

这是一个简单的示例，在训练过程中保存了每批次损失值的列表：

In [79]:
class LossHistory(keras.callbacks.Callback):

    def on_train_begin(self, logs):
        self.losses = []

    def on_batch_end(self, batch, logs):
        self.losses.append(logs.get('loss'))

### 检查点模型
在相对较大的数据集上训练模型时，至关重要的是要定期保存模型的检查点。

最简单的方法是使用ModelCheckpoint回调：

In [ ]:
model = get_compiled_model()

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='mymodel_{epoch}',
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]
model.fit(x_train, y_train,
          epochs=3,
          batch_size=64,
          callbacks=callbacks,
          validation_split=0.2)

您还调用了自己的回调函数来保存和恢复模型。

有关序列化和保存的完整指南，请参阅《模型的保存和序列化指南》。

### 使用学习率时间表
训练深度学习模型的常见模式是随着训练的进行逐渐减少学习。这通常称为“学习率衰减”。

学习衰减进度表可以是静态的（根据当前纪元或当前批次索引预先确定），也可以是动态的（响应于模型的当前行为，尤其是验证损失）。

将时间表传递给优化器
通过将计划对象作为learning_rate优化器中的参数传递，可以轻松使用静态学习率衰减计划：

In [80]:
initial_learning_rate = 0.1
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

optimizer = keras.optimizers.RMSprop(learning_rate=lr_schedule)

内置多种时间表可供选择：ExponentialDecay，PiecewiseConstantDecay，PolynomialDecay，和InverseTimeDecay。
### 使用回调实现动态学习率计划
由于优化程序无法访问验证指标，因此无法使用这些计划对象来实现动态学习率计划（例如，当验证损失不再改善时降低学习率）。

但是，回调确实可以访问所有指标，包括验证指标！因此，您可以通过使用回调来修改优化程序上的当前学习率，从而实现此模式。实际上，它甚至是作为ReduceLROnPlateau回调内置的。

### 可视化训练期间的损失和指标
在训练过程中密切关注模型的最好方法是使用TensorBoard，这是一个基于浏览器的应用程序，您可以在本地运行该程序，为您提供：

实时损失图以及用于评估和评估的指标
（可选）可视化图层激活的直方图
（可选）您的Embedding图层所学习的嵌入空间的3D可视化
如果您已通过pip安装TensorFlow，则应该能够从命令行启动TensorBoard：

In [ ]:
tensorboard --logdir=/full_path_to_your_logs

### 使用TensorBoard回调
将TensorBoard与Keras模型一起使用的最简单fit方法是TensorBoard回调。

在最简单的情况下，只需指定您希望回调写日志的位置，就可以了：

In [ ]:
tensorboard_cbk = keras.callbacks.TensorBoard(log_dir='/full_path_to_your_logs')
model.fit(dataset, epochs=10, callbacks=[tensorboard_cbk])

In [ ]:
该TensorBoard回调有许多有用的选项，包括是否记录的嵌入，直方图，以及如何经常写日志：

In [ ]:
keras.callbacks.TensorBoard(
  log_dir='/full_path_to_your_logs',
  histogram_freq=0,  # How often to log histogram visualizations
  embeddings_freq=0,  # How often to log embedding visualizations
  update_freq='epoch')  # How often to write logs (default: once per epoch)

## 第二部分：从头开始编写自己的培训和评估循环
如果你想低级别在你的训练和评估环比fit()和evaluate()提供，你应该写你自己的。实际上很简单！但是您应该准备自己进行更多的调试。

### 使用GradientTape：第一个端到端示例
GradientTape在合并范围内调用模型使您能够检索层的可训练权重相对于损耗值的梯度。使用优化程序实例，您可以使用这些渐变来更新这些变量（可以使用进行检索model.trainable_weights）。

让我们重用第一部分中的初始MNIST模型，并使用带有训练循环的小批量梯度对其进行训练。

In [81]:
# Get the model.
inputs = keras.Input(shape=(32,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, name='predictions')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

# Instantiate an optimizer.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the training dataset.
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

为几个时期运行训练循环：

In [82]:
epochs = 3
for epoch in range(epochs):
    print('Start of epoch %d' % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables autodifferentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = model(x_batch_train, training=True)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            loss_value = loss_fn(y_batch_train, logits)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, model.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Log every 200 batches.
        if step % 200 == 0:
            print('Training loss (for one batch) at step %s: %s' % (step, float(loss_value)))
            print('Seen so far: %s samples' % ((step + 1) * 64))


Start of epoch 0

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Training loss (for one batch) at step 0: 2.294208288192749
Seen so far: 64 samples
Start of epoch 1
Training loss (for one batch) at step 0: 2.3047895431518555
Seen so far: 64 samples
Start of epoch 2
Training loss (for one batch) at step 0: 2.3294360637664795
Seen so far: 64 samples


## 指标的低级处理
让我们将指标添加到组合中。您可以在从头开始编写的训练循环中随时使用内置指标（或您编写的自定义指标）。流程如下：

在循环开始时实例化指标
metric.update_state()每批之后致电
metric.result()需要显示指标的当前值时调用
metric.reset_states()需要清除指标状态时致电（通常在纪元末尾）
让我们使用这些知识SparseCategoricalAccuracy在每个时期结束时计算验证数据：

In [83]:
# Get model
inputs = keras.Input(shape=(32,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, name='predictions')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

# Prepare the training dataset.
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(64)

为几个时期运行训练循环：

In [84]:
epochs = 3
for epoch in range(epochs):
    print('Start of epoch %d' % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update training metric.
        train_acc_metric(y_batch_train, logits)

        # Log every 200 batches.
        if step % 200 == 0:
            print('Training loss (for one batch) at step %s: %s' % (step, float(loss_value)))
            print('Seen so far: %s samples' % ((step + 1) * 64))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print('Training acc over epoch: %s' % (float(train_acc),))
    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val)
        # Update val metrics
        val_acc_metric(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print('Validation acc: %s' % (float(val_acc),))


Start of epoch 0

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Training loss (for one batch) at step 0: 2.3608527183532715
Seen so far: 64 samples
Training acc over epoch: 0.10300000011920929
Validation acc: 0.11999999731779099
Start of epoch 1
Training loss (for one batch) at step 0: 2.358880043029785
Seen so far: 64 samples
Training acc over epoch: 0.10300000011920929
Validation acc: 0.11999999731779099
Start of epoch 2
Training loss (for one batch) at step 0: 2.3007380962371826
Seen so far: 64 samples
Training acc over epoch: 0.10300000011920929

### 低水平处理额外损失
在上一节中，您已经看到有可能通过调用方法将正则化损失添加到一个层self.add_loss(value)中call。

在一般情况下，您将需要在训练循环中考虑这些损失（除非您自己编写模型并且您已经知道它不会造成这种损失）。

回顾上一节中的示例，其中包含一个会产生正则化损失的层：



In [87]:
class ActivityRegularizationLayer(layers.Layer):

    def call(self, inputs):
        self.add_loss(1e-2 * tf.reduce_sum(inputs))
        return inputs


inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
# Insert activity regularization as a layer
x = ActivityRegularizationLayer()(x)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs)


调用模型时，如下所示：

In [ ]:
logits = model(x_train)

它在前向传递过程中产生的损失将添加到model.losses属性中：

In [ ]:
logits = model(x_train[:64])
print(model.losses)

跟踪的损失首先在模型开始时清除`__call__`，因此您只会看到在此前向传递过程中产生的损失。例如，反复调用模型然后查询losses仅显示最新的损失，该损失是在上一次调用期间创建的：

In [ ]:
logits = model(x_train[:64])
logits = model(x_train[64: 128])
logits = model(x_train[128: 192])
print(model.losses)

要在训练过程中考虑这些损失，您要做的就是修改训练循环以增加sum(model.losses)总损失：

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=1e-3)

epochs = 3
for epoch in range(epochs):
    print('Start of epoch %d' % (epoch,))

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train)
            loss_value = loss_fn(y_batch_train, logits)

            # Add extra losses created during this forward pass:
            loss_value += sum(model.losses)

        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Log every 200 batches.
        if step % 200 == 0:
            print('Training loss (for one batch) at step %s: %s' % (step, float(loss_value)))
            print('Seen so far: %s samples' % ((step + 1) * 64))
